In [105]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

In [115]:
lifeExpectancyPrediction_Train=pd.read_csv('train.csv')
lifeExpectancyPrediction_Test=pd.read_csv('test.csv')

from sklearn.model_selection import KFold 
X_Train = lifeExpectancyPrediction_Train.drop(['ID','TARGET_LifeExpectancy'], axis=1)
Y_Train=  lifeExpectancyPrediction_Train[['TARGET_LifeExpectancy']]

X_Test = lifeExpectancyPrediction_Test.drop('ID', axis=1)


# kf = KFold(n_splits=5, shuffle=True)
kf = KFold(n_splits=7, shuffle=True)
# kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(X_Train)

print(kf)

KFold(n_splits=10, random_state=None, shuffle=True)


In [116]:
from sklearn.preprocessing import MinMaxScaler
scaledSet_minMax = MinMaxScaler()
X_Train_Scaled_minMax=scaledSet_minMax.fit_transform(X_Train)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [117]:
scaledSet_minMax.fit(Y_Train)
Y_Train_Scaled_minMax = scaledSet_minMax.fit_transform(Y_Train)

In [123]:
from sklearn.preprocessing import StandardScaler

scaledSet = StandardScaler()
scaledSet.fit(X_Train)
X_Train_Scaled_standardScaler = scaledSet.fit_transform(X_Train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [124]:
scaledSet.fit(Y_Train)
Y_Train_Scaled_standardScaler = scaledSet.fit_transform(Y_Train)

In [125]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

linReg = linear_model.LinearRegression()



In [126]:
for train_index, validation_index in kf.split(X_Train_Scaled_standardScaler):
    trainX = np.array(X_Train_Scaled_minMax[train_index])
    trainY = np.array(Y_Train_Scaled_minMax[train_index])
    validX = np.array(X_Train_Scaled_minMax[validation_index])
    validY = np.array(Y_Train_Scaled_minMax[validation_index])
    fittedModel=linReg.fit(trainX, trainY)
    print(linReg.intercept_)
    print(linReg.coef_)
    predYRm=linReg.predict(validX)
    print('Mean squared error ', mean_squared_error(validY, predYRm))
    print('Mean Absolute error ', mean_absolute_error(validY, predYRm))
    print('R2 ', r2_score(validY, predYRm))
    MSEs=mean_squared_error(validY, predYRm)
    MAEs=mean_absolute_error(validY, predYRm)
    R2s=r2_score(validY, predYRm)
    predictedTestSet=fittedModel.predict(validX)
#     predictedTestSet


[0.31956934]
[[ 9.05236754e-03  1.77693081e-02  4.19268519e-02 -9.05722903e-02
  -8.54035515e-03 -1.67259038e-01  3.02890717e+00  6.21995254e-02
   1.06526665e-02  8.73712678e-03  5.20750809e-02 -3.13432347e+00
   3.09530365e-02 -2.06726225e-03  5.27046287e-02 -4.53353654e-01
   1.00869277e-01 -3.52417893e-02 -3.68909765e-02 -1.67305040e-02
   1.06916772e-01  1.81399963e-01]]
Mean squared error  0.005818984806228773
Mean Absolute error  0.06042520596442526
R2  0.7723933582671253
[0.31936948]
[[ 1.14173915e-02  1.35012509e-02  4.44483669e-02 -4.89790387e+12
   2.47000310e+12  2.52613953e+12  2.91417474e+00  5.84829041e-02
   3.96627164e-03 -4.27451928e-02  5.17346779e-02 -3.00252509e+00
   4.11342206e-02 -3.74471111e-03  5.51333025e-02 -4.50524177e-01
   9.26888618e-02 -5.21764554e-02 -2.66416384e-02 -2.73797171e-02
   1.03535334e-01  1.76743278e-01]]
Mean squared error  0.00932665876747135
Mean Absolute error  0.07384094954894495
R2  0.6840003959375482
[0.31625068]
[[ 1.01375109e-02  1

## R2 means Main Dataset

In [127]:
mean_MSE = np.mean(MSEs)
mean_absolute=np.mean(MAEs)
mean_R2_score=np.mean(R2s)


print(mean_MSE)
print(mean_absolute)
print(mean_R2_score)



0.0055367304452422815
0.05706231816060231
0.8042986050171756


In [114]:
predictedResults=pd.DataFrame(predictedTestSet, columns=['Predicted Life Expectancy']).to_csv('Linear_Prediction_KFOLD.csv')
